<a href="https://colab.research.google.com/github/Veewy/AML/blob/main/AML_3)_ModelClass_Training_Loop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌹 **Anti Money Laundering**  🌹

# **Data Import**

In [1]:
!pip install torchmetrics

In [2]:
!pip install optuna

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import torch
import torch.nn as nn
import torch.optim as optim
import optuna
import os

from torch.optim.lr_scheduler import ReduceLROnPlateau
from functools import partial
from torch.utils.data import Dataset, DataLoader
from datetime import datetime
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
from torch.nn.utils.rnn import pad_sequence #pad the sequences to the same lengt
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.metrics import confusion_matrix, precision_score, recall_score
from torchmetrics import F1Score, AUROC
from torch.nn.utils.rnn import pad_sequence # to support batch >1 in dataloader, so need to match the longest sequence in the batch/limit max history length.
from torch.nn.utils.rnn import pack_sequence

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **DataSet** / **DataLoader**

In [5]:
class AmlDataset(Dataset):
    def __init__(self, data, features):
        """
        @param data: pdf whose index is monotonically increases from 0
        @param features: list of features to be used in an event
        """
        self.data = data
        self.features = features

    def __getitem__(self, index):
        psr_sample = self.data.iloc[index] #retrieves row at specified index from the data.
        list_y = psr_sample["targets"]
        list_x = []

        for event in psr_sample["events"]:
          x = [event[feature] for feature in self.features] #create X
          list_x.append(x)

        #Converts the extracted features (list_x) and targets (list_y) to NumPy arrays, Returns a tuple (x, y).
        return np.array(list_x).astype(np.float32), np.array(list_y).astype(np.float32)

    def __len__(self):
        return len(self.data)

Load Dataset/DataLoader

## a) Sampled Dataset

In [6]:
# for training part

aml_dataset_sampled = torch.load("/content/drive/MyDrive/DE/Master_Degree/3rd_Semester/Colab_Notebook/AML_file/aml_dataset_sampled.pth")
aml_dataloader_sampled= torch.load( "/content/drive/MyDrive/DE/Master_Degree/3rd_Semester/Colab_Notebook/AML_file/aml_dataloader_sampled.pth")


<ipython-input-6-9d08c78ddfa8>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  aml_dataset_sampled = torch.load("/content/drive/MyDrive/DE/Master_Degree/3rd_Semester/Colab_

## b) Downsampled Negative Class

In [7]:
# for training part

aml_dataset_sampled_downsampled= torch.load("/content/drive/MyDrive/DE/Master_Degree/3rd_Semester/Colab_Notebook/AML_file/aml_dataset_sampled_downsampled.pth")
aml_dataloader_sampled_downsampled= torch.load("/content/drive/MyDrive/DE/Master_Degree/3rd_Semester/Colab_Notebook/AML_file/aml_dataloader_sampled_downsampled.pth")


<ipython-input-7-fc7bd0fb4f27>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  aml_dataset_sampled_downsampled= torch.load("/content/drive/MyDrive/DE/Master_Degree/3rd_Seme

## a+b) both scenarios use the same path for eval_part

In [8]:
# for evaluating both sampled & and downsampled dataset

aml_eval_dataset_sampled = torch.load("/content/drive/MyDrive/DE/Master_Degree/3rd_Semester/Colab_Notebook/AML_file/aml_eval_dataset_sampled.pth")
aml_eval_dataloader_sampled = torch.load("/content/drive/MyDrive/DE/Master_Degree/3rd_Semester/Colab_Notebook/AML_file/aml_eval_dataloader_sampled.pth")


<ipython-input-8-a5861d4b8966>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  aml_eval_dataset_sampled = torch.load("/content/drive/MyDrive/DE/Master_Degree/3rd_Semester/C

## c) Sampled Dataset (with a.c.interaction)

In [19]:
# for training part
#aml_dataset_sampled_id
#aml_dataloader_sampled_id
aml_dataset_sampled_id = torch.load("/content/drive/MyDrive/DE/Master_Degree/3rd_Semester/Colab_Notebook/AML_file/aml_dataset_sampled_id.pth")
aml_dataloader_sampled_id = torch.load("/content/drive/MyDrive/DE/Master_Degree/3rd_Semester/Colab_Notebook/AML_file/aml_dataloader_sampled_id.pth")

<ipython-input-19-d67d26b6c743>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  aml_dataset_sampled_id = torch.load("/content/drive/MyDrive/DE/Master_Degree/3rd_Semester/Co

In [7]:
aml_dataset_sampled_id.data.iloc[0].events

array([{'account': 1323989368.0, 'account_encoded': 0.0, 'account_interaction': 1603624323.0, 'account_interaction_encoded': 6832.0, 'amount_normalized': 0.5174716092738527, 'day_of_week_encoded': 6.0, 'hour': 4.0, 'payment_currency_encoded': 7.0, 'received_currency_encoded': 10.0, 'receiver_bank_location_encoded': 16.0, 'sender_bank_location_encoded': 11.0, 'time_interval_normalized': -0.054420741832058306, 'transaction_type_encoded': 0.0},
       {'account': 1323989368.0, 'account_encoded': 0.0, 'account_interaction': 1603624323.0, 'account_interaction_encoded': 6832.0, 'amount_normalized': 0.5230400101024983, 'day_of_week_encoded': 6.0, 'hour': 5.0, 'payment_currency_encoded': 7.0, 'received_currency_encoded': 10.0, 'receiver_bank_location_encoded': 16.0, 'sender_bank_location_encoded': 11.0, 'time_interval_normalized': -0.021537644082431154, 'transaction_type_encoded': 0.0},
       {'account': 1323989368.0, 'account_encoded': 0.0, 'account_interaction': 1603624323.0, 'account_inter

In [20]:
# for evaluating part
#aml_eval_dataset_sampled_id
#aml_eval_dataloader_sampled_id

aml_eval_dataset_sampled_id = torch.load("/content/drive/MyDrive/DE/Master_Degree/3rd_Semester/Colab_Notebook/AML_file/aml_eval_dataset_sampled_id.pth")
aml_eval_dataloader_sampled_id = torch.load("/content/drive/MyDrive/DE/Master_Degree/3rd_Semester/Colab_Notebook/AML_file/aml_eval_dataloader_sampled_id.pth")

<ipython-input-20-f61b8270e695>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  aml_eval_dataset_sampled_id = torch.load("/content/drive/MyDrive/DE/Master_Degree/3rd_Semest

## c.1) For small overfitting test

In [21]:
pdf_tiny_train = aml_dataset_sampled_id.data.copy()
pdf_tiny_train["is_positive"] = pdf_tiny_train["targets"].apply(lambda x: 1 if 1 in x else 0)
pdf_tiny_train["is_positive"].value_counts()

,count
is_positive,
0,7922
1,78


In [22]:
#down to 700 sample
pdf_tiny_train_neg = pdf_tiny_train[pdf_tiny_train.is_positive == 0]
pdf_tiny_train_neg = pdf_tiny_train_neg.sample(n=700, random_state=42, replace=False)
pdf_tiny_train_neg

,events,targets,is_positive
account,,,
6588974294,"[{'account': 6588974294.0, 'account_encoded': ...","[0, 0, 0, 0, 0, 0, 0, 0]",0
5215570935,"[{'account': 5215570935.0, 'account_encoded': ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0
1616849948,"[{'account': 1616849948.0, 'account_encoded': ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0
6968424815,"[{'account': 6968424815.0, 'account_encoded': ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0
8683935386,"[{'account': 8683935386.0, 'account_encoded': ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0
...,...,...,...
5766818513,"[{'account': 5766818513.0, 'account_encoded': ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0
245982509,"[{'account': 245982509.0, 'account_encoded': 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0
5142072942,"[{'account': 5142072942.0, 'account_encoded': ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0


In [23]:
pdf_tiny_train_combined = pd.concat([pdf_tiny_train_neg, pdf_tiny_train[pdf_tiny_train.is_positive == 1]])
pdf_tiny_train_combined

,events,targets,is_positive
account,,,
6588974294,"[{'account': 6588974294.0, 'account_encoded': ...","[0, 0, 0, 0, 0, 0, 0, 0]",0
5215570935,"[{'account': 5215570935.0, 'account_encoded': ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0
1616849948,"[{'account': 1616849948.0, 'account_encoded': ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0
6968424815,"[{'account': 6968424815.0, 'account_encoded': ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0
8683935386,"[{'account': 8683935386.0, 'account_encoded': ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0
...,...,...,...
8718560017,"[{'account': 8718560017.0, 'account_encoded': ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",1
5591631486,"[{'account': 5591631486.0, 'account_encoded': ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
111267387,"[{'account': 111267387.0, 'account_encoded': 4...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",1


In [56]:
aml_dataset_id_tiny = AmlDataset(
    pdf_tiny_train_combined,
     ['account_encoded','day_of_week_encoded', 'hour', 'transaction_type_encoded','account_interaction_encoded',
      'payment_currency_encoded', 'received_currency_encoded',
      'sender_bank_location_encoded', 'receiver_bank_location_encoded',
      'time_interval_normalized', 'amount_normalized'
      ]
    )

aml_dataloader_id_tiny = DataLoader(
    aml_dataset_id_tiny,
    batch_size= None,
    shuffle=True,
    num_workers=1,
    pin_memory=False, # if we have GPU, set pin_memory=True
    drop_last=False, # every sample in dataset is important, even if the final batch size varies. So will not drop it.
)

In [25]:
# for evaluating part
#aml_eval_dataset_sampled_id
#aml_eval_dataloader_sampled_id

pdf_tiny_eval = aml_eval_dataset_sampled_id.data.copy()
pdf_tiny_eval["is_positive"] = pdf_tiny_eval["targets"].apply(lambda x: 1 if 1 in x else 0)
pdf_tiny_eval["is_positive"].value_counts()

,count
is_positive,
0,1976
1,24


In [27]:
pdf_tiny_eval_neg = pdf_tiny_eval[pdf_tiny_eval.is_positive == 0]
pdf_tiny_eval_neg = pdf_tiny_eval_neg.sample(n=250, random_state=42, replace=False)
pdf_tiny_eval_neg

,events,targets,is_positive
account,,,
6626179956,"[{'account': 6626179956.0, 'account_encoded': ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
6887309800,"[{'account': 6887309800.0, 'account_encoded': ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0
365844412,"[{'account': 365844412.0, 'account_encoded': 0...","[0, 0, 0, 0, 0, 0, 0, 0]",0
375974960,"[{'account': 375974960.0, 'account_encoded': 0...","[0, 0, 0, 0]",0
4971847802,"[{'account': 4971847802.0, 'account_encoded': ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0
...,...,...,...
6226836565,"[{'account': 6226836565.0, 'account_encoded': ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0
7681160265,"[{'account': 7681160265.0, 'account_encoded': ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0
6340787367,"[{'account': 6340787367.0, 'account_encoded': ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0


In [28]:
pdf_tiny_eval_combined = pd.concat([pdf_tiny_eval_neg, pdf_tiny_eval[pdf_tiny_eval.is_positive == 1]])
pdf_tiny_eval_combined

,events,targets,is_positive
account,,,
6626179956,"[{'account': 6626179956.0, 'account_encoded': ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
6887309800,"[{'account': 6887309800.0, 'account_encoded': ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0
365844412,"[{'account': 365844412.0, 'account_encoded': 0...","[0, 0, 0, 0, 0, 0, 0, 0]",0
375974960,"[{'account': 375974960.0, 'account_encoded': 0...","[0, 0, 0, 0]",0
4971847802,"[{'account': 4971847802.0, 'account_encoded': ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0
...,...,...,...
6251401807,"[{'account': 6251401807.0, 'account_encoded': ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
1574251475,"[{'account': 1574251475.0, 'account_encoded': ...",[1],1
5590746929,"[{'account': 5590746929.0, 'account_encoded': ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1


In [63]:
aml_dataset_id_tiny_eval = AmlDataset(
    pdf_tiny_eval_combined,
     ['account_encoded','day_of_week_encoded', 'hour', 'transaction_type_encoded','account_interaction_encoded',
      'payment_currency_encoded', 'received_currency_encoded',
      'sender_bank_location_encoded', 'receiver_bank_location_encoded',
      'time_interval_normalized', 'amount_normalized'
      ]
    )

aml_dataloader_id_tiny_eval = DataLoader(
    aml_dataset_id_tiny_eval,
    batch_size= None,
    shuffle=True,
    num_workers=1,
    pin_memory=False, # if we have GPU, set pin_memory=True
    drop_last=False
)

## d) Full Dataset (w/o a.c.interaction)

In [11]:
# for training part

aml_dataset_noid = torch.load("/content/drive/MyDrive/DE/Master_Degree/3rd_Semester/Colab_Notebook/AML_file/aml_dataset_noid.pth")
aml_dataloader_noid = torch.load("/content/drive/MyDrive/DE/Master_Degree/3rd_Semester/Colab_Notebook/AML_file/aml_dataloader_noid.pth")

<ipython-input-11-7e89adf4fc41>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  aml_dataset_noid = torch.load("/content/drive/MyDrive/DE/Master_Degree/3rd_Semester/Colab_No

KeyboardInterrupt: 

In [ ]:
# for evaluating part

aml_eval_dataset_noid = torch.load("/content/drive/MyDrive/DE/Master_Degree/3rd_Semester/Colab_Notebook/AML_file/aml_eval_dataset_noid.pth")
aml_eval_dataloader_noid = torch.load("/content/drive/MyDrive/DE/Master_Degree/3rd_Semester/Colab_Notebook/AML_file/aml_eval_dataloader_noid.pth")

## e) Full Dataset (with a.c.interaction)



In [6]:
"""
note:
cat_features = [['day_of_week_encoded', 3], ['hour', 4],
['transaction_type_encoded', 3],['account_interaction_encoded', 95],
['payment_currency_encoded', 2],['received_currency_encoded', 2],
['sender_bank_location_encoded', 3],['receiver_bank_location_encoded', 3]]


- 'account'                          cat 0-43557,  ---> size ~35
- 'day_of_week_encoded':             cat  0-6. ----> root 3 size~ 2
- 'hour':                            cat   0-23 ----> size ~ 3
- 'transaction_type_encoded':        cat   0-1  ----> size ~ 1
- 'time_interval_normalized':        numerical
- 'amount_normalized':               numerical
- 'account_interaction_encoded'      cat 0-43557,  ---> size ~35
- 'payment_currency_encoded':        cat   0-12 ----> size ~ 2
- 'received_currency_encoded':       cat   0-12 ----> size ~ 2
- 'sender_bank_location_encoded':    cat   0-17 ----> size ~ 3
- 'receiver_bank_location_encoded':  cat   0-17 ----> size ~ 3
"""

" \nnote:\ncat_features = [['day_of_week_encoded', 3], ['hour', 4],\n['transaction_type_encoded', 3],['account_interaction_encoded', 95],\n['payment_currency_encoded', 2],['received_currency_encoded', 2],\n['sender_bank_location_encoded', 3],['receiver_bank_location_encoded', 3]]\n\n\n- 'account'                          ignored\n- 'day_of_week_encoded':             cat  0-6. ----> root 3 size~ 2\n- 'hour':                            cat   0-23 ----> size ~ 3\n- 'transaction_type_encoded':        cat   0-1  ----> size ~ 1\n- 'time_interval_normalized':        numerical\n- 'amount_normalized':               numerical\n- 'account_interaction_encoded'      cat   0-855459 ---> size ~ 95\n- 'payment_currency_encoded':        cat   0-12 ----> size ~ 2\n- 'received_currency_encoded':       cat   0-12 ----> size ~ 2\n- 'sender_bank_location_encoded':    cat   0-17 ----> size ~ 3\n- 'receiver_bank_location_encoded':  cat   0-17 ----> size ~ 3\n"

In [6]:
# for training part

aml_dataset_id = torch.load("/content/drive/MyDrive/DE/Master_Degree/3rd_Semester/Colab_Notebook/AML_file/aml_dataset_id.pth")
aml_dataloader_id = torch.load("/content/drive/MyDrive/DE/Master_Degree/3rd_Semester/Colab_Notebook/AML_file/aml_dataloader_id.pth")

<ipython-input-6-b3b18165740a>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  aml_dataset_id = torch.load("/content/drive/MyDrive/DE/Master_Degree/3rd_Semester/Colab_Noteb

KeyboardInterrupt: 

In [ ]:
# for evaluating part

aml_eval_dataset_id = torch.load("/content/drive/MyDrive/DE/Master_Degree/3rd_Semester/Colab_Notebook/AML_file/aml_eval_dataset_id.pth")
aml_eval_dataloader_id = torch.load("/content/drive/MyDrive/DE/Master_Degree/3rd_Semester/Colab_Notebook/AML_file/aml_eval_dataloader_id.pth")

## f) Down negative class_Full Dataset (with a.c. interaction)
deal with imbalanced dataset

1)

In [7]:
# # for training part
# #aml_dataset_id_downs2
# #aml_dataloader_id_downs2

aml_dataset_id_downs2 = torch.load( "/content/drive/MyDrive/DE/Master_Degree/3rd_Semester/Colab_Notebook/AML_file/aml_dataset_id_downs2.pth")
aml_dataloader_id_downs2 = torch.load( "/content/drive/MyDrive/DE/Master_Degree/3rd_Semester/Colab_Notebook/AML_file/aml_dataloader_id_downs2.pth")

# train_dataset_downsampled = pd.read_parquet("/content/drive/MyDrive/DE/Master_Degree/3rd_Semester/Colab_Notebook/AML_file/train_dataset_downsampled.parquet")
# train_dataset_downsampled

<ipython-input-7-3e32cc0b19fc>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  aml_dataset_id_downs1 = torch.load( "/content/drive/MyDrive/DE/Master_Degree/3rd_Semester/Col

In [8]:
# for evaluating part
#aml_eval_dataset_id2
#aml_eval_dataloader_id2


aml_eval_dataset_id2 = torch.load( "/content/drive/MyDrive/DE/Master_Degree/3rd_Semester/Colab_Notebook/AML_file/aml_eval_dataset_id2.pth")
aml_eval_dataloader_id2 = torch.load( "/content/drive/MyDrive/DE/Master_Degree/3rd_Semester/Colab_Notebook/AML_file/aml_eval_dataloader_id2.pth")

<ipython-input-8-34fbef946bba>:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  aml_eval_dataset_id1 = torch.load( "/content/drive/MyDrive/DE/Master_Degree/3rd_Semester/Cola

In [9]:
# for testing part
#aml_test_dataset_id2
#aml_test_dataloader_id2

aml_test_dataset_id2 = torch.load( "/content/drive/MyDrive/DE/Master_Degree/3rd_Semester/Colab_Notebook/AML_file/aml_test_dataset_id2.pth")
aml_test_dataloader_id2 = torch.load( "/content/drive/MyDrive/DE/Master_Degree/3rd_Semester/Colab_Notebook/AML_file/aml_test_dataloader_id2.pth")

<ipython-input-9-c19fe8d2641f>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  aml_test_dataset_id1 = torch.load( "/content/drive/MyDrive/DE/Master_Degree/3rd_Semester/Cola

# Model Class

In [10]:
class AmlLstm(nn.Module):
    def __init__(
        self, cat_features, total_numerical_features, hidden_size,
        fc_input_layers, fc_output_layers, dropout_prob=0.1):
        """
        @param cat_features: A list of (n_classes, emb_size) tuples for cat features
        @param total_numerical_features:
        @param hidden_size:
        @param fc_input_layers: A list of FC sizes to be placed between input
          vector and the LSTM
        @param fc_output_layers: A list of FC sizes to be placed between the
          LSTM output to the final Sigmoid output. The last size should be 1.
        """
        super(AmlLstm, self).__init__()
        embedded_features = []
        in_fc_layers = []
        output_layers = []
        embedding_output_dim = 0

        for [n_classes, emb_size] in cat_features:
            emb_layer = nn.Embedding(n_classes, emb_size)
            embedded_features.append(emb_layer)
            embedding_output_dim += emb_size #The sum of all emb_size values determines the total embedding dimension

        prev_size = embedding_output_dim + total_numerical_features

        for current_size in fc_input_layers:
            in_fc_layers.append(nn.Dropout(p=dropout_prob))
            in_fc_layers.append(nn.Linear(prev_size, current_size, bias=True))
            prev_size = current_size

        lstm = nn.LSTM(prev_size, hidden_size, batch_first=True)
        prev_size = hidden_size

        for current_size in fc_output_layers:
            output_layers.append(nn.Dropout(p=dropout_prob))
            output_layers.append(nn.Linear(prev_size, current_size, bias=True))
            prev_size = current_size

        output_layers.append(nn.Sigmoid()) # to produce probabilities

        self.total_numerical_features = total_numerical_features
        self.embedded_features = nn.ModuleList(embedded_features)
        self.in_fc_layers = nn.ModuleList(in_fc_layers)
        self.lstm_layer = nn.ModuleList([lstm])
        self.output_layers = nn.ModuleList(output_layers)
        total_trainable_params = sum(p.numel() for p in self.parameters() if p.requires_grad)
        print(f'Total trainable params: {total_trainable_params}')

    def forward(self, x):
        """
        @param x: a Tensor of shape (tot_events, event_features) for a SINGLE user
            [
              [c1, c2, ..., cC, n1, n2, ..., nN],
              [...],
               ...
              [...]
            ]
            where C = len(cat_features), N = numerical_features
        @return: a Tensor of shape [batch_size, output_size]
        """
        # prepare the input vector to LSTM (categorical and numerical features)
        vector_input = []

        for cat_feat_idx, embedded_layer in enumerate(self.embedded_features):
            cat_input = x[:, cat_feat_idx].type(torch.IntTensor)
            vector_input.append(embedded_layer(cat_input))  # shape (tot_events, emb_size)

        total_cat_features = len(self.embedded_features)

        for num_feat_idx in range(self.total_numerical_features):
            num_input = x[:, total_cat_features + num_feat_idx].unsqueeze(1)  # shape (tot_events, 1)
            vector_input.append(num_input)

        vector_input_tensor = torch.cat(vector_input, dim=1)  # shape (tot_events, embedding_output_dim + total_numerical_features)

        for fc_layer in self.in_fc_layers:
            vector_input_tensor = fc_layer(vector_input_tensor)

        # pass to LSTM
        lstm_input_tensor = vector_input_tensor
        out_tensor, (h, c) = self.lstm_layer[0](lstm_input_tensor) # no previous state

        # make predict
        for output_layer in self.output_layers:
            out_tensor = output_layer(out_tensor)

        return out_tensor

# Training Loop

### Define Evaluation Metrics

In [11]:
def compute_metrics(all_preds, all_targets, threshold=0.5):

    # Convert probabilities to binary predictions
    binary_preds = (all_preds >= threshold).float()

    # Compute confusion matrix components
    TP = ((binary_preds == 1) & (all_targets == 1)).sum().item()
    TN = ((binary_preds == 0) & (all_targets == 0)).sum().item()
    FP = ((binary_preds == 1) & (all_targets == 0)).sum().item()
    FN = ((binary_preds == 0) & (all_targets == 1)).sum().item()

    # Metrics
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    TNR = TN / (TN + FP) if (TN + FP) > 0 else 0
    FPR = FP / (FP + TN) if (FP + TN) > 0 else 0
    FNR = FN / (FN + TP) if (FN + TP) > 0 else 0
    accuracy = (TP + TN) / (TP + TN + FP + FN) if (TP + TN + FP + FN) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    try:
        auc = roc_auc_score(all_targets.cpu().numpy(), all_preds.cpu().numpy())
    except ValueError:
        auc = None #if AUC cannot be computed

    return precision, recall, TNR, FPR, FNR, accuracy, f1, auc

In [53]:
# Just to test compute_metrics() function
# all_preds = torch.tensor([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])
# all_targets = torch.tensor([0, 0, 0, 0, 0, 0, 0, 1, 1])
# precision, recall, TNR, FPR, FNR, accuracy, f1, auc = compute_metrics(all_preds, all_targets)
# precision, recall, TNR, FPR, FNR, accuracy, f1, auc

(0.4,
 1.0,
 0.5714285714285714,
 0.42857142857142855,
 0.0,
 0.6666666666666666,
 0.5714285714285715,
 1.0)

### Define The Loss fn. and Optimizer

In [12]:
# Define the weighted BCE loss function
def compute_weighted_BCELoss(pred, Y, criterion, positive_weight=100):
    """
    Compute BCELoss with weights for positive samples being larger than the
    negative ones.
    """
    Y_expanded = Y[:, None]
    weights = (Y_expanded * (positive_weight - 1)) + 1
    loss = criterion(pred, Y_expanded) #compute the unweighted binary cross-entropy loss for each sample
    weighted_loss = loss * weights #applying weights
    final_loss = torch.mean(weighted_loss) # reduce loss across the batch
    return final_loss

## Define Training Utilities

In [13]:
def train_one_epoch(
        epoch, dataloader, optimizer, model, criterion, positive_weight
    ):
    total_loss = 0
    total_samples = 0
    total_training_events = 0
    total_positives_training_Y = 0
    total_positives_training_pred = 0

    for X, Y in dataloader:
        optimizer.zero_grad()
        pred = model(X)
        loss = compute_weighted_BCELoss(
            pred, Y, criterion, positive_weight=positive_weight
        )
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_samples += 1
        total_training_events += len(Y)
        total_positives_training_Y += Y.sum().item()
        total_positives_training_pred += pred.sum().item()

        if total_samples % 10000 == 0:
            print(f"===> {100 * total_samples / len(dataloader):.2f}% completed")

    print(f'Epoch: {epoch}, Training loss: {total_loss / total_samples}, \
      positive stats {total_positives_training_pred / total_training_events}: \
      {total_positives_training_Y / total_training_events}'
    )


def eval_one_epoch(dataloader, model, criterion, positive_weight):
    total_eval_events = 0
    total_positives_eval_Y = 0
    total_positives_eval_pred = 0
    all_preds = []
    all_targets = []
    total_val_loss = 0

    with torch.no_grad():
        for X, Y in dataloader:
            pred = model(X)
            val_loss = compute_weighted_BCELoss(
                pred, Y, criterion, positive_weight=positive_weight
            )
            total_val_loss += val_loss.item()
            all_preds.append(pred)
            all_targets.append(Y)

    all_preds = torch.squeeze(torch.cat(all_preds), dim=1)
    all_targets = torch.squeeze(torch.cat(all_targets))
    total_eval_events += len(all_targets)
    total_positives_eval_pred += all_preds.sum().item()
    total_positives_eval_Y += all_targets.sum().item()
    avg_val_loss = total_val_loss / total_eval_events

    # Compute metrics
    precision, recall, TNR, FPR, FNR, accuracy, f1, auc = compute_metrics(all_preds, all_targets)
    positive_ratio_eval_pred = total_positives_eval_pred / total_eval_events
    positive_ratio_eval_Y = total_positives_eval_Y / total_eval_events

    print(f"Metrics:")
    print(f"Validation loss: {avg_val_loss}")
    print(f"Precision: {precision:.3f}, Recall (TPR): {recall:.3f}, TNR: {TNR:.3f}")
    print(f"FPR: {FPR:.3f}, FNR: {FNR:.3f}, Accuracy: {accuracy:.3f}, F1 Score: {f1:.3f}, AUC: {auc:.3f}")
    print(f"positive stats {positive_ratio_eval_pred }: {positive_ratio_eval_Y}")

    return avg_val_loss, precision, recall, TNR, FPR, FNR, accuracy, f1, auc, \
      positive_ratio_eval_pred, positive_ratio_eval_Y, all_preds, all_targets


def train_and_evaluate(
        epochs, dataloader_train, dataloader_eval, model, optimizer, criterion,
        positive_weight, lr_scheduler, patience=5, save_path=None
    ):
    """
    Train and evaluate the model for a specified number of epochs, with early stopping and intermediate saving.
    Includes gradient accumulation for memory efficiency.
    """
    log_results = []
    best_val_loss = float('inf') # Track the best validation loss
    no_improve_count = 0 # Track epochs with no improvement

    for epoch in range(epochs):
        # Train and evaluate for a complete epoch
        train_one_epoch(
            epoch, dataloader_train, optimizer, model, criterion,
            positive_weight
        )
        avg_val_loss, precision, recall, TNR, FPR, FNR, accuracy, f1, auc, \
        positive_ratio_eval_pred, positive_ratio_eval_Y, all_preds, \
        all_targets = eval_one_epoch(
            dataloader_eval, model, criterion, positive_weight
        )

        # Update LR scheduler
        lr_scheduler.step(avg_val_loss)
        print(f"LR: {lr_scheduler.get_last_lr()}")

        # Log metrics
        epoch_results = (
            epoch, avg_val_loss, precision, recall, TNR, FPR, FNR, accuracy, f1, auc,\
            positive_ratio_eval_pred, positive_ratio_eval_Y
        )
        log_results.append(epoch_results)

        # Save log and model every few epochs
        if (epoch % 1 == 0) and (save_path is not None):
            df_log = pd.DataFrame(
                log_results,
                columns=[
                    "epoch", "val_loss", "precision", "recall", "TNR", "FPR",
                    "FNR", "accuracy", "f1", "auc","positive_ratio_eval_pred",
                    "positive_ratio_eval_Y"
                ]
            )
            df_log_path = os.path.join(save_path, f"df_log_epoch_{epoch}.parquet")
            df_log.to_parquet(df_log_path)
            print(f"Saved intermediate results to {df_log_path}")

            model_path = os.path.join(save_path, f"model_epoch_{epoch}.bin")
            torch.save(model, model_path)
            print(f"Saved model to {model_path}")

        # Early stopping
        if avg_val_loss < best_val_loss - 1e-4:  # Improvement threshold
            best_val_loss = avg_val_loss
            no_improve_count = 0
        else:
            no_improve_count += 1

        if no_improve_count >= patience:
            print(f"======== *** Early stopping triggered at epoch {epoch + 1} *** ========")
            break

        print("\n===================================\n")


    return pd.DataFrame(
        log_results,
        columns=[
            "epoch", "val_loss", "precision", "recall", "TNR", "FPR",
                    "FNR", "accuracy", "f1", "auc", "positive_ratio_eval_pred",
                    "positive_ratio_eval_Y"
        ]
    ), all_preds, all_targets

## Overfitting test

In [75]:
hidden_size = 19
positive_weight = 59
total_numerical_features = 2
cat_features = [[43557, 50],[7, 4],[24, 6],[2, 2],[43557, 50],[13, 4],[13, 4],[18, 5],[18, 5]]
fc_input_layers = [100, 90, 80, 70, 60, 50, 40, 40, 30, 30, 20, 10, 10]
fc_output_layers = [50, 45, 40, 30, 25, 20, 15, 10, 5, 1]
dropout_prob = 0
model = AmlLstm(cat_features, total_numerical_features, hidden_size, fc_input_layers, fc_output_layers, dropout_prob=dropout_prob)
criterion = nn.BCELoss(weight= None, reduction='none') # define BCE loss criterion with no reduction
optimizer = optim.Adam(model.parameters(), lr=0.001) # small lerning rate for stable training
lr_scheduler = ReduceLROnPlateau(optimizer, mode="min", factor=0.1, patience=3, threshold=1e-4, min_lr=1e-7)
epochs = 100
save_path = "/content/drive/MyDrive/DE/Master_Degree/3rd_Semester/Colab_Notebook/AML_file/exp/overfitting"
final_file_name = "df_log_final.parquet"

df_log = train_and_evaluate(
     epochs, aml_dataloader_id_tiny, aml_dataloader_id_tiny, model, optimizer,
     criterion, positive_weight, lr_scheduler, patience=100, save_path=save_path
)

df_log
df_log.to_parquet(os.path.join(save_path, final_file_name), engine="pyarrow", index=False)

Total trainable params: 4416142
Epoch: 0, Training loss: 1.2620864993121774,       positive stats 0.37912815656283144:       0.006630384792273494
Metrics:
Validation loss: 0.030974700405026402
Precision: 0.000, Recall (TPR): 0.000, TNR: 1.000
FPR: 0.000, FNR: 1.000, Accuracy: 0.993, F1 Score: 0.000, AUC: 0.499
positive stats 0.4682673703266327: 0.006630384792273494
LR: [0.001]
Saved intermediate results to /content/drive/MyDrive/DE/Master_Degree/3rd_Semester/Colab_Notebook/AML_file/exp/overfitting/df_log_epoch_0.parquet
Saved model to /content/drive/MyDrive/DE/Master_Degree/3rd_Semester/Colab_Notebook/AML_file/exp/overfitting/model_epoch_0.bin


Epoch: 1, Training loss: 0.9320899403934154,       positive stats 0.2785345428006042:       0.006630384792273494
Metrics:
Validation loss: 0.02501207382450911
Precision: 0.000, Recall (TPR): 0.000, TNR: 1.000
FPR: 0.000, FNR: 1.000, Accuracy: 0.993, F1 Score: 0.000, AUC: 0.746
positive stats 0.09657322452164456: 0.006630384792273494
LR: [0.001]

## Hyperparameter Tuning

### for scenario a/ b/ d (w/o a.c.interaction)

In [ ]:
"""
note:
cat_features = [['day_of_week_encoded', 3], ['hour', 4],
['transaction_type_encoded', 3],
['payment_currency_encoded', 2],['received_currency_encoded', 2],
['sender_bank_location_encoded', 3],['receiver_bank_location_encoded', 3]]


- 'account'                          ignored
- 'day_of_week_encoded':             cat   0-6 ----> root 3 size~ 2
- 'hour':                            cat   0-23 ----> size ~ 3
- 'transaction_type_encoded':        cat   0-1  ----> size ~ 1
- 'time_interval_normalized':        numerical
- 'amount_normalized':               numerical
- 'account_interaction_encoded'      ---- ignored ----
- 'payment_currency_encoded':        cat   0-12 ----> size ~ 2
- 'received_currency_encoded':       cat   0-12 ----> size ~ 2
- 'sender_bank_location_encoded':    cat   0-17 ----> size ~ 3
- 'receiver_bank_location_encoded':  cat   0-17 ----> size ~ 3
"""

In [ ]:
2+3+1+2+2+3+3+2

In [15]:
def objective(trial, hp_dataloader, hp_eval_dataloader, epochs):
    # Hyperparameter ranges to search
    hidden_size = trial.suggest_int("hidden_size", 5, 50)
    positive_weight = trial.suggest_int("positive_weight", 2, 100)
    cat_features = trial.suggest_categorical(
        "cat_features",
        [
            [[7, 2],[24, 3],[2, 1],[13, 2],[13, 2],[18, 3],[18, 3]],
            [[7, 3],[24, 4],[2, 1],[13, 3],[13, 3],[18, 4],[18, 4]],
        ]
    )
    fc_input_layers = trial.suggest_categorical(
        "fc_input_layers",
        [
            [18, 16, 14],
            [20, 18, 16, 14],
            [18, 18, 16, 16, 14, 14],
            [20, 20, 18, 18, 16, 16, 14, 14],
        ]
    )
    fc_output_layers = trial.suggest_categorical(
        "fc_output_layers",
        [
            [10, 5, 1],
            [15, 10, 5, 1],
            [20, 15, 10, 5, 1],
            [30, 25, 20, 15, 10, 5, 1],
            [40, 35, 30, 25, 20, 15, 10, 5, 1],
        ]
    )
    dropout_prob = trial.suggest_float("dropout_prob", 0.0, 0.3)

    # Define the model and other configurable training objects
    total_numerical_features = 2
    model = AmlLstm(
        cat_features, total_numerical_features, hidden_size, fc_input_layers,
        fc_output_layers, dropout_prob=dropout_prob
    )
    criterion = nn.BCELoss(weight=None, reduction='none')
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    lr_scheduler = ReduceLROnPlateau(
        optimizer, mode="min", factor=0.1, patience=3, threshold=1e-4,
        min_lr=1e-7
    )

    # Train and evaluate
    df_log, _, _  = train_and_evaluate(
        epochs, hp_dataloader, hp_eval_dataloader, model,
        optimizer, criterion, positive_weight, lr_scheduler,
        patience=10
    )

    # Calculate the average of the last 3 F1 scores
    last_3_f1_scores = df_log.f1.iloc[-3:]  # Get the last 3 F1 scores
    avg_last_3_f1 = last_3_f1_scores.mean()  # Calculate their average

    print(f"===>  Average of the last 3 F1 scores: {avg_last_3_f1:.8f}")

    # Return the average of the last 3 F1 metrics
    return avg_last_3_f1

### for scenario c/e/f (with a.c.interaction)

In [33]:
"""
note:
cat_features = [['day_of_week_encoded', xx], ['hour', xx],
['transaction_type_encoded', xx],['account_interaction_encoded', 95],
['payment_currency_encoded', xx],['received_currency_encoded', xx],
['sender_bank_location_encoded', xx],['receiver_bank_location_encoded', xx]]


- 'account'                          cat 0-43557,  ---> size ~35
- 'day_of_week_encoded':             cat   0-6  ----> root 3 size~ 2
- 'hour':                            cat   0-23 ----> size ~ 3
- 'transaction_type_encoded':        cat   0-1  ----> size ~ 1
- 'time_interval_normalized':        numerical
- 'amount_normalized':               numerical
- 'account_interaction_encoded'      cat   0-43557,  ---> size ~35
- 'payment_currency_encoded':        cat   0-12 ----> size ~ 2
- 'received_currency_encoded':       cat   0-12 ----> size ~ 2
- 'sender_bank_location_encoded':    cat   0-17 ----> size ~ 3
- 'receiver_bank_location_encoded':  cat   0-17 ----> size ~ 3
"""

" \nnote:\ncat_features = [['day_of_week_encoded', xx], ['hour', xx],\n['transaction_type_encoded', xx],['account_interaction_encoded', 95],\n['payment_currency_encoded', xx],['received_currency_encoded', xx],\n['sender_bank_location_encoded', xx],['receiver_bank_location_encoded', xx]]\n\n\n- 'account'                          cat 0-43557,  ---> size ~35\n- 'day_of_week_encoded':             cat   0-6  ----> root 3 size~ 2\n- 'hour':                            cat   0-23 ----> size ~ 3\n- 'transaction_type_encoded':        cat   0-1  ----> size ~ 1\n- 'time_interval_normalized':        numerical\n- 'amount_normalized':               numerical\n- 'account_interaction_encoded'      cat   0-43557,  ---> size ~35\n- 'payment_currency_encoded':        cat   0-12 ----> size ~ 2\n- 'received_currency_encoded':       cat   0-12 ----> size ~ 2\n- 'sender_bank_location_encoded':    cat   0-17 ----> size ~ 3\n- 'receiver_bank_location_encoded':  cat   0-17 ----> size ~ 3\n"

In [38]:
def objective(trial, hp_dataloader, hp_eval_dataloader, epochs):
    # Hyperparameter ranges to search
    hidden_size = trial.suggest_int("hidden_size", 5, 90)
    positive_weight = trial.suggest_int("positive_weight", 2, 100)
    cat_features = trial.suggest_categorical(
        "cat_features",
        [
            [[43557, 35],[7, 2],[24, 3],[2, 1],[43557, 35],[13, 2],[13, 2],[18, 3],[18, 3]],
            [[43557, 40],[7, 3],[24, 4],[2, 1],[43557, 40],[13, 3],[13, 3],[18, 4],[18, 4]],
            [[43557, 50],[7, 4],[24, 6],[2, 2],[43557, 50],[13, 4],[13, 4],[18, 5],[18, 5]],

        ]
    )
    fc_input_layers = trial.suggest_categorical(
        "fc_input_layers",
        [
            [80, 40, 15],
            [85, 55, 35, 20, 15],
            [90, 70, 50, 30, 20, 10],
            [95, 75, 60, 50, 40, 30, 20, 10],
            [100, 80, 60, 50, 40, 40, 30, 30, 20, 10, 10],
            [100, 90, 80, 70, 60, 50, 40, 40, 30, 30, 20, 10, 10],
        ]
    )
    fc_output_layers = trial.suggest_categorical(
        "fc_output_layers",
        [
            [10, 7, 1],
            [15, 10, 5, 3, 1],
            [18, 15, 10, 8, 4, 1],
            [18, 16, 12, 7, 4, 2, 1],
            [20, 18, 16, 12, 7, 4, 2, 1],
            [25, 20, 18, 16, 12, 7, 4, 2, 1],
            [35, 25, 20, 18, 16, 12, 7, 4, 2, 1],
            [50, 45, 40, 30, 25, 20, 15, 10, 5, 1],
        ]
    )
    dropout_prob = trial.suggest_float("dropout_prob", 0.0, 0.5)

    # Define the model and other configurable training objects
    total_numerical_features = 2
    model = AmlLstm(
        cat_features, total_numerical_features, hidden_size, fc_input_layers,
        fc_output_layers, dropout_prob=dropout_prob
    )
    criterion = nn.BCELoss(weight=None, reduction='none')
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    lr_scheduler = ReduceLROnPlateau(
        optimizer, mode="min", factor=0.1, patience=3, threshold=1e-4,
        min_lr=1e-7
    )

    # Train and evaluate
    df_log, _, _ = train_and_evaluate(
        epochs, hp_dataloader, hp_eval_dataloader, model,
        optimizer, criterion, positive_weight, lr_scheduler,
        patience=10
    )

    # Calculate the average of the last 3 F1 scores
    last_3_f1_scores = df_log.f1.iloc[-3:]  # Get the last 3 F1 scores
    avg_last_3_f1 = last_3_f1_scores.mean()  # Calculate their average

    print(f"===>  Average of the last 3 F1 scores: {avg_last_3_f1:.8f}")

    # Return the average of the last 3 F1 metrics
    return avg_last_3_f1

## Optuna

In [65]:
%%time

study = optuna.create_study(direction="maximize")
study.optimize(
    partial(
        objective,
        hp_dataloader=aml_dataloader_id_tiny,             #<< revise everytime
        hp_eval_dataloader=aml_dataloader_id_tiny_eval,   #<< revise everytime
        epochs=50
    ),
    n_trials=40
)

print("Best hyperparameters:", study.best_params)
print("Best avr last3 F1:", study.best_value)
#print(f"Best F1: {study.best_value}")

[I 2024-12-02 09:09:33,461] A new study created in memory with name: no-name-c8a5ada8-ec57-41ab-bec9-022d71a7ef25


Total trainable params: 3547879
Epoch: 0, Training loss: 1.877948686638949,       positive stats 0.33849525760901356:       0.006630384792273494
Metrics:
Validation loss: 0.02437983717981728
Precision: 0.009, Recall (TPR): 0.277, TNR: 0.868
FPR: 0.132, FNR: 0.723, Accuracy: 0.865, F1 Score: 0.018, AUC: 0.581
positive stats 0.46954632808176533: 0.004334993543626637
LR: [0.001]


Epoch: 1, Training loss: 1.2672162461154413,       positive stats 0.41254316965780813:       0.006630384792273494
Metrics:
Validation loss: 0.023980493844380085
Precision: 0.004, Recall (TPR): 0.149, TNR: 0.831
FPR: 0.169, FNR: 0.851, Accuracy: 0.828, F1 Score: 0.007, AUC: 0.489
positive stats 0.40423558088625255: 0.004334993543626637
LR: [0.001]


Epoch: 2, Training loss: 1.2748438953817043,       positive stats 0.32637934988441963:       0.006630384792273494
Metrics:
Validation loss: 0.024344793584948864
Precision: 0.006, Recall (TPR): 0.362, TNR: 0.757
FPR: 0.243, FNR: 0.638, Accuracy: 0.755, F1 Score: 0.013,

[I 2024-12-02 09:14:07,732] Trial 0 finished with value: 0.01080356510695613 and parameters: {'hidden_size': 37, 'positive_weight': 99, 'cat_features': [[43557, 40], [7, 3], [24, 4], [2, 1], [43557, 40], [13, 3], [13, 3], [18, 4], [18, 4]], 'fc_input_layers': [100, 90, 80, 70, 60, 50, 40, 40, 30, 30, 20, 10, 10], 'fc_output_layers': [50, 45, 40, 30, 25, 20, 15, 10, 5, 1], 'dropout_prob': 0.2831129601424672}. Best is trial 0 with value: 0.01080356510695613.


Metrics:
Validation loss: 0.024310518808634028
Precision: 0.006, Recall (TPR): 0.553, TNR: 0.604
FPR: 0.396, FNR: 0.447, Accuracy: 0.604, F1 Score: 0.012, AUC: 0.607
positive stats 0.4345769778408043: 0.004334993543626637
LR: [1e-05]
======== *** Early stopping triggered at epoch 17 *** ========
===>  Average of the last 3 F1 scores: 0.01080357
Total trainable params: 3513656


/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [[43557, 35], [7, 2], [24, 3], [2, 1], [43557, 35], [13, 2], [13, 2], [18, 3], [18, 3]] which is of type list.
  warnings.warn(message)
/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [[43557, 40], [7, 3], [24, 4], [2, 1], [43557, 40], [13, 3], [13, 3], [18, 4], [18, 4]] which is of type list.
  warnings.warn(message)
/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [[43557, 50], [7, 4], [24, 6], [2, 2], [43557, 50], [13, 4], [13, 4], [18, 5], [18, 5]] which is of t

...

Metrics:
Validation loss: 0.01529061176526032
Precision: 0.077, Recall (TPR): 0.426, TNR: 0.978
FPR: 0.022, FNR: 0.574, Accuracy: 0.975, F1 Score: 0.130, AUC: 0.870
positive stats 0.062229254306891024: 0.004334993543626637
LR: [1.0000000000000002e-06]
======== *** Early stopping triggered at epoch 25 *** ========
===>  Average of the last 3 F1 scores: 0.13588236
Total trainable params: 3075030


/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [[43557, 35], [7, 2], [24, 3], [2, 1], [43557, 35], [13, 2], [13, 2], [18, 3], [18, 3]] which is of type list.
  warnings.warn(message)
/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [[43557, 40], [7, 3], [24, 4], [2, 1], [43557, 40], [13, 3], [13, 3], [18, 4], [18, 4]] which is of type list.
  warnings.warn(message)
/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [[43557, 50], [7, 4], [24, 6], [2, 2], [43557, 50], [13, 4], [13, 4], [18, 5], [18, 5]] which is of t

Epoch: 0, Training loss: 1.2883356640780952,       positive stats 0.19584100796662188:       0.006630384792273494
Metrics:
Validation loss: 0.019323441509368388
Precision: 0.000, Recall (TPR): 0.000, TNR: 1.000
FPR: 0.000, FNR: 1.000, Accuracy: 0.996, F1 Score: 0.000, AUC: 0.520
positive stats 0.27223535174264435: 0.004334993543626637
LR: [0.001]


Epoch: 1, Training loss: 1.0812004060223508,       positive stats 0.20574562984461453:       0.006630384792273494
Metrics:
Validation loss: 0.017357962237273476
Precision: 0.024, Recall (TPR): 0.149, TNR: 0.974
FPR: 0.026, FNR: 0.851, Accuracy: 0.970, F1 Score: 0.041, AUC: 0.464
positive stats 0.3937584307553957: 0.004334993543626637
LR: [0.001]


Epoch: 2, Training loss: 0.8365394158533039,       positive stats 0.2952717283517054:       0.006630384792273494
Metrics:
Validation loss: 0.020786085750669203
Precision: 0.061, Recall (TPR): 0.085, TNR: 0.994
FPR: 0.006, FNR: 0.915, Accuracy: 0.990, F1 Score: 0.071, AUC: 0.420
positive stats 0.206

[I 2024-12-02 12:42:44,642] Trial 39 finished with value: 0.17452418028969455 and parameters: {'hidden_size': 40, 'positive_weight': 83, 'cat_features': [[43557, 35], [7, 2], [24, 3], [2, 1], [43557, 35], [13, 2], [13, 2], [18, 3], [18, 3]], 'fc_input_layers': [85, 55, 35, 20, 15], 'fc_output_layers': [18, 16, 12, 7, 4, 2, 1], 'dropout_prob': 0.0029469561732347856}. Best is trial 34 with value: 0.24933078144936593.


Metrics:
Validation loss: 0.024482208071319494
Precision: 0.131, Recall (TPR): 0.234, TNR: 0.993
FPR: 0.007, FNR: 0.766, Accuracy: 0.990, F1 Score: 0.168, AUC: 0.676
positive stats 0.0684873731873876: 0.004334993543626637
LR: [1e-05]
======== *** Early stopping triggered at epoch 12 *** ========
===>  Average of the last 3 F1 scores: 0.17452418
Best hyperparameters: {'hidden_size': 30, 'positive_weight': 50, 'cat_features': [[43557, 35], [7, 2], [24, 3], [2, 1], [43557, 35], [13, 2], [13, 2], [18, 3], [18, 3]], 'fc_input_layers': [85, 55, 35, 20, 15], 'fc_output_layers': [18, 16, 12, 7, 4, 2, 1], 'dropout_prob': 0.006260916610701425}
Best avr last3 F1: 0.24933078144936593
CPU times: user 12h 54min 15s, sys: 22min 46s, total: 13h 17min 2s
Wall time: 3h 33min 11s


## Train on the best hyperparameters

In [20]:
hidden_size = 30
positive_weight = 50
total_numerical_features = 2
cat_features = [[43557, 35],[7, 2],[24, 3],[2, 1],[43557, 35],[13, 2],[13, 2],[18, 3],[18, 3]]
fc_input_layers = [85, 55, 35, 20, 15]
fc_output_layers = [18, 16, 12, 7, 4, 2, 1]
dropout_prob = 0.006260916610701425
model = AmlLstm(cat_features, total_numerical_features, hidden_size, fc_input_layers, fc_output_layers, dropout_prob=dropout_prob)
criterion = nn.BCELoss(weight= None, reduction='none') # define BCE loss criterion with no reduction
optimizer = optim.Adam(model.parameters(), lr=0.001) # small lerning rate for stable training
lr_scheduler = ReduceLROnPlateau(optimizer, mode="min", factor=0.1, patience=3, threshold=1e-4, min_lr=1e-7)
epochs = 50
save_path = "/content/drive/MyDrive/DE/Master_Degree/3rd_Semester/Colab_Notebook/AML_file/exp/3full_t_downs" #<< revise everytime
final_file_name = "df_log_final.parquet"

df_log, all_preds, all_targets = train_and_evaluate(
     epochs, aml_dataloader_id_downs1, aml_eval_dataloader_id1, model, optimizer,     #<< name of dataloder revise everytime
     criterion, positive_weight, lr_scheduler, patience=5, save_path=save_path
)

df_log
df_log.to_parquet(os.path.join(save_path, final_file_name), engine="pyarrow", index=False)

Total trainable params: 3071370
===> 5.69% completed
===> 11.38% completed
===> 17.06% completed
===> 22.75% completed
===> 28.44% completed
===> 34.13% completed
===> 39.82% completed
===> 45.51% completed
===> 51.19% completed
===> 56.88% completed
===> 62.57% completed
===> 68.26% completed
===> 73.95% completed
===> 79.64% completed
===> 85.32% completed
===> 91.01% completed
===> 96.70% completed
Epoch: 0, Training loss: 0.4001820080643792,       positive stats 0.10035103973133329:       0.0033555893796802104
Metrics:
Validation loss: 0.007940612309269625
Precision: 0.001, Recall (TPR): 0.005, TNR: 0.994
FPR: 0.006, FNR: 0.995, Accuracy: 0.993, F1 Score: 0.002, AUC: 0.475
positive stats 0.054883978582660795: 0.001105176290173756
LR: [0.001]
Saved intermediate results to /content/drive/MyDrive/DE/Master_Degree/3rd_Semester/Colab_Notebook/AML_file/exp/3full_t_downs/df_log_epoch_0.parquet
Saved model to /content/drive/MyDrive/DE/Master_Degree/3rd_Semester/Colab_Notebook/AML_file/exp/

### *IN CASE of section disconnected while runing code* !!!

In [21]:
# in case of disconnection; download the intermediate file to continue runing
path = "/content/drive/MyDrive/DE/Master_Degree/3rd_Semester/Colab_Notebook/AML_file/exp/1" #change ourselves


model_path = os.path.join(path, f"model_epoch_0.bin") #change according to the last runnning epoch, eg. 0
model = torch.load(model_path)

<ipython-input-21-1c40cd350286>:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_path)


In [22]:
#tell model to continue run from above
df_log = pd.read_parquet(os.path.join(path, "df_log_epoch_0.parquet"))